In [14]:
import numpy as np
import pydae.build as db
import sympy as sym
from pydae.models import grid2dae_dq,pydgrid2pydae
import json

!['cigre_lv'](./cigre_lv.png)
<img src="cigre_lv_loads.png" alt="Drawing" style="width: 200px;"/>

In [15]:
sin = sym.sin
cos = sym.cos
sqrt = sym.sqrt

In [19]:
Omega_b = 2*np.pi*50
L_t = 1.25e-3; #Converter side Filter Inductance H
R_t = L_t*Omega_b/10; #Converter side Filter resistance Ohms. Quality Factor of 10
L_s = 1.25e-3; #Grid Side Filter Inductance H
R_s = L_s*Omega_b/10; #Grid side Filter resistance Ohms. Quality Factor of 10
R_d = 20; #Dmaping resistor
C_m = 4e-6; #Filter capacitance
tau_ctrl_1 = 5e-3; #Time constant of CTRL 1
Ts_Power=50e-6; #Sample time of power plant
Ts_Control=50e-6; #Sample time of controller
K_pi = (L_t+L_s)/tau_ctrl_1; #Proportional gain of CTRL 1
K_ii = (R_t+R_s)/tau_ctrl_1; #Integral gain of CTRL 1
H = 5.0; # desired virtual inertia 
K_p = 0.01; # active power proportinal gain
#H = T_p/K_p/2

T_p = K_p*2*H;  # active power integral time constant

K_q = 0.1; # reactive power proportinal gain
T_q = 0.1; # reactive power integral time constant

X_v = 0.05; # virtual reactance
R_v = 0.01;  # virtual resistance

K_pv = 0.0004
K_iv = 11.313708

G10 = {"name":"G10","bus": "R10","U_b":400.0,"S_b_kVA":200,"v_dc":750,'type':'sm2_i',
           'R_t':R_t,'L_t':L_t,'R_s':R_s,'L_s':L_s,'C_m':C_m,'K_phi':0.00,'omega':Omega_b, 'G_d':1e-2, 'eta_D':0,'eta_Q':0.8, #VSC
           'eta_d_ref':0,'eta_q_ref':0.8,'phi':0.0,   # CTRL0
           'K_pi':K_pi,'K_ii':K_ii,'Gv_in':0.1,   # CTRL1
           'K_pv':K_pv,'K_iv':K_iv,   # CTRL2
           'K_p':K_p,'T_p':T_p,'K_q':K_q,'T_q':T_q,'R_v':R_v,'X_v':X_v,'p_m_ref':0.0,'q_s_ref':0.0,   # CTRL3
           'H':5.0,'D':5.0,  # CTRL3
           'T_vpoi':0.1,'K_vpoi':10,'T_f':0.1,'K_f':20.0,'p_r':0.0,'q_r':0.0,'v_s_ref':1.0,'omega_ref':1.0}   # CTRL4

G14 = {"name":"G14","bus": "R14","U_b":400.0,"S_b_kVA":200,"v_dc":750,'type':'sm2_i',
           'R_t':R_t,'L_t':L_t,'R_s':R_s,'L_s':L_s,'C_m':C_m,'K_phi':0.00,'omega':Omega_b, 'G_d':1e-2, 'eta_D':0,'eta_Q':0.8, #VSC
           'eta_d_ref':0,'eta_q_ref':0.8,'phi':0.0,   # CTRL0
           'K_pi':K_pi,'K_ii':K_ii,'Gv_in':0.1,   # CTRL1
           'K_pv':K_pv,'K_iv':K_iv,   # CTRL2
           'K_p':K_p,'T_p':T_p,'K_q':K_q,'T_q':T_q,'R_v':R_v,'X_v':X_v,'p_m_ref':0.0,'q_s_ref':0.0,   # CTRL3
           'H':5.0,'D':5.0,  # CTRL3
           'T_vpoi':0.1,'K_vpoi':10,'T_f':0.1,'K_f':20.0,'p_r':0.0,'q_r':0.0,'v_s_ref':1.0,'omega_ref':1.0}   # CTRL4

data = {
"system":{"f_hz":50,"model_type":"ae"},
"buses": [{"bus":"R00"},{"bus":"R01"},{"bus":"R02"},{"bus":"R03"},{"bus":"R04"},{"bus":"R05"},{"bus":"R06"},
        {"bus":"R07"},{"bus":"R08"},{"bus":"R09"},{"bus":"R10"},{"bus":"R11"},{"bus":"R12"},
        {"bus":"R13"},{"bus":"R14"},{"bus":"R15"},{"bus":"R16"},{"bus":"R17"},{"bus":"R18"}],
"lines":     [  {"bus_j":"R00", "bus_k":"R01", "R":0.0032, "X":0.0128, "B":0.0},
                {"bus_j":"R02", "bus_k":"R01", "R":0.00567210, "X":0.002839550, "B":0.0},
                {"bus_j":"R02", "bus_k":"R03", "R":0.00567210, "X":0.002839550, "B":0.0},
                {"bus_j":"R03", "bus_k":"R04", "R":0.00567210, "X":0.002839550, "B":0.0},
                {"bus_j":"R04", "bus_k":"R05", "R":0.00567210, "X":0.002839550, "B":0.0},
                {"bus_j":"R04", "bus_k":"R12", "R":0.02877350, "X":0.002983400, "B":0.0},
                {"bus_j":"R05", "bus_k":"R06", "R":0.00567210, "X":0.002839550, "B":0.0},
                {"bus_j":"R06", "bus_k":"R07", "R":0.00567210, "X":0.002839550, "B":0.0},
                {"bus_j":"R07", "bus_k":"R08", "R":0.00567210, "X":0.002839550, "B":0.0},
                {"bus_j":"R08", "bus_k":"R09", "R":0.00567210, "X":0.002839550, "B":0.0},
                {"bus_j":"R09", "bus_k":"R10", "R":0.00567210, "X":0.002839550, "B":0.0},
                {"bus_j":"R09", "bus_k":"R17", "R":0.02466300, "X":0.002557200, "B":0.0},
                {"bus_j":"R11", "bus_k":"R03", "R":0.02466300, "X":0.002557200, "B":0.0},
                {"bus_j":"R12", "bus_k":"R13", "R":0.02877350, "X":0.002983400, "B":0.0},
                {"bus_j":"R13", "bus_k":"R14", "R":0.02877350, "X":0.002983400, "B":0.0},
                {"bus_j":"R14", "bus_k":"R15", "R":0.02877350, "X":0.002983400, "B":0.0},
                {"bus_j":"R16", "bus_k":"R06", "R":0.02466300, "X":0.002557200, "B":0.0},
                {"bus_j":"R18", "bus_k":"R10", "R":0.02466300, "X":0.002557200, "B":0.0}],
"loads" : [
        {"bus": "R01", "kVA":  1.0, "pf": 0.95, "T_i":0.01,"I_max":434.78},
        {"bus": "R11", "kVA": 15.0, "pf": 0.95, "T_i":0.01,"I_max":32.6},
        {"bus": "R15", "kVA": 52.0, "pf": 0.95, "T_i":0.01,"I_max":120},
        {"bus": "R16", "kVA": 55.0, "pf": 0.95, "T_i":0.01,"I_max":120},
        {"bus": "R17", "kVA": 35.0, "pf": 0.95, "T_i":0.01,"I_max":120},
        {"bus": "R18", "kVA": 47.0, "pf": 0.95, "T_i":0.01,"I_max":120}
        ],
"grid_formers" : [
    {"bus": "R00","V_phph":400.0, "deg":0.0}
],
#"vsgs" : [G10,G14]
}

grid_dae = pydgrid2pydae(data)

AttributeError: 'dict' object has no attribute 'buses'

In [17]:
i_R01_D,i_R01_Q  = sym.symbols('i_R01_D,i_R01_Q', real=True)
#f_vsg = [indum-dum]
#x_vsg = [      dum]
#u_run_dict = {'indum':1.0}

params_dict = grid_dae['params'] 
u_run_dict = grid_dae['u']
f_list = grid_dae['f'] 
x_list = grid_dae['x'] 
g_list = grid_dae['g']
y_list = grid_dae['y']
h_dict = {'i_R01': (i_R01_D**2 + i_R01_Q**2)**0.5}
#params_dict.update(params_vsg)
#u_run_dict.update(u_vsg) 
#params_dict.update(params_vsg)


#u_run_dict.pop('v_1_D')
#u_run_dict.pop('v_1_Q')
#u_run_dict.pop('v_2_D')
#u_run_dict.pop('v_2_Q')
#u_run_dict.pop('v_2_D')
#u_run_dict.pop('v_2_Q')
#u_run_dict.pop('v_R10_d')
#u_run_dict.pop('v_R10_q')

#params_dict.pop('i_1_D')
#params_dict.pop('i_1_Q')
#params_dict.pop('i_3_D')
#params_dict.pop('i_3_Q')

sys = {'name':'cigre_lv_res',
       'params_dict':params_dict,
       'f_list':f_list,
       'g_list':g_list,
       'x_list':x_list,
       'y_ini_list':y_list,
       'y_run_list':y_list,
       'u_run_dict':u_run_dict,
       'u_ini_dict':u_run_dict,
       'h_dict':h_dict
      }

sys = db.system(sys)
db.sys2num(sys)


In [18]:
sys

{'name': 'cigre_lv_res',
 'params_dict': {'R_R00R01': 0.0032,
  'L_R00R01': 4.074366543152521e-05,
  'C_R00R01': 0.0,
  'R_R02R01': 0.0056721,
  'L_R02R01': 9.038568373131828e-06,
  'C_R02R01': 0.0,
  'R_R02R03': 0.0056721,
  'L_R02R03': 9.038568373131828e-06,
  'C_R02R03': 0.0,
  'R_R03R04': 0.0056721,
  'L_R03R04': 9.038568373131828e-06,
  'C_R03R04': 0.0,
  'R_R04R05': 0.0056721,
  'L_R04R05': 9.038568373131828e-06,
  'C_R04R05': 0.0,
  'R_R04R12': 0.0287735,
  'L_R04R12': 9.496457144407212e-06,
  'C_R04R12': 0.0,
  'R_R05R06': 0.0056721,
  'L_R05R06': 9.038568373131828e-06,
  'C_R05R06': 0.0,
  'R_R06R07': 0.0056721,
  'L_R06R07': 9.038568373131828e-06,
  'C_R06R07': 0.0,
  'R_R07R08': 0.0056721,
  'L_R07R08': 9.038568373131828e-06,
  'C_R07R08': 0.0,
  'R_R08R09': 0.0056721,
  'L_R08R09': 9.038568373131828e-06,
  'C_R08R09': 0.0,
  'R_R09R10': 0.0056721,
  'L_R09R10': 9.038568373131828e-06,
  'C_R09R10': 0.0,
  'R_R09R17': 0.024663,
  'L_R09R17': 8.139820409491894e-06,
  'C_R09R17

### VSGs

In [9]:
xi_f_sec = sym.Symbol('xi_f_sec', real=True)
K_f_sec = sym.Symbol('K_f_sec', real=True)
omega_v_G10,omega_v_G14 = sym.symbols('omega_v_G10,omega_v_G14', real=True)
H_G10,H_G14 = sym.symbols('H_G10,H_G14', real=True)
omega_coi = (H_G10*omega_v_G10 + H_G14*omega_v_G14)/(H_G10+H_G14)
dxi_f_sec = 1 - omega_coi
p_sec_G10 = K_f_sec*xi_f_sec/2
p_sec_G14 = K_f_sec*xi_f_sec/2

f_ctrl_5 = [dxi_f_sec]
x_ctrl_5 = [xi_f_sec]
g_ctrl_5 = []
y_ctrl_5 = []
x_0_ctrl_5 = []
y_0_ctrl_5 = []
params_ctrl_5 = {'K_f_sec':0.001}
u_ctrl_5 = {}
h_ctrl_5 = {}

p_sec = {'G10':p_sec_G10,'G14':p_sec_G14}

#omega_coi = 1.0
grid_dae = grid2dae_dq(data)

vsgs = data['vsgs']
f_vsg = []
x_vsg = []
g_vsg = []
y_vsg = []
x_0_vsg = []
y_0_vsg = []
params_vsg = {}
u_vsg = {}
h_vsg = {}

for vsg in vsgs:
    name = vsg['name']
    bus = vsg['bus']
    
    U_b = vsg['U_b']
    S_b = vsg['S_b_kVA']*1000
    I_b = S_b/(np.sqrt(3)*U_b)
 
    U_bdq = U_b*(np.sqrt(2))
    V_bdq = U_bdq/np.sqrt(3)
    I_bdq = I_b*np.sqrt(2)

           

            
    ## Transformations #########################################################################
    ## feedbacks:
    feedbacks = ['i_tD','i_tQ'] + ['v_mD','v_mQ'] + ['i_sD','i_sQ'] + ['v_sD','v_sQ'] + ['phi']
    for item in feedbacks:
        exec(f"{item} = sym.Symbol('{item}_{name}', real=True)")    

    
    
    v_md,v_mq = sym.symbols(f'v_md_{name},v_mq_{name}', real=True)
    v_sd,v_sq = sym.symbols(f'v_sd_{name},v_sq_{name}', real=True)
    i_sd,i_sq = sym.symbols(f'i_sd_{name},i_sq_{name}', real=True)
    i_td,i_tq = sym.symbols(f'i_td_{name},i_tq_{name}', real=True)
    phi,dum= sym.symbols(f'phi_{name},dum_{name}', real=True)

    eq_v_md = -v_md + v_mD*cos(phi) + v_mQ*sin(phi) # original park
    eq_v_mq = -v_mq - v_mD*sin(phi) + v_mQ*cos(phi) # original park
    
    eq_v_sd = -v_sd + v_sD*cos(phi) + v_sQ*sin(phi) # original park
    eq_v_sq = -v_sq - v_sD*sin(phi) + v_sQ*cos(phi) # original park

    eq_i_sd = -i_sd + i_sD*cos(phi) + i_sQ*sin(phi) # original park
    eq_i_sq = -i_sq - i_sD*sin(phi) + i_sQ*cos(phi) # original park

    # jmm: medimos i_t?
    eq_i_td = -i_td + i_tD*cos(phi) + i_tQ*sin(phi) # original park
    eq_i_tq = -i_tq - i_tD*sin(phi) + i_tQ*cos(phi) # original park

    g_aux = [eq_v_md,eq_v_mq,eq_v_sd,eq_v_sq,eq_i_td,eq_i_tq,eq_i_sd,eq_i_sq]    
    y_aux = [   v_md,   v_mq,   v_sd,   v_sq,   i_td,   i_tq,   i_sd,   i_sq]
    y_0_aux = [  0.0,  V_bdq,    0.0,  V_bdq,       0,      0]    


    #v_sd = v_md
    #v_sq = v_mq


   # S_b_kVA,U_b = sym.symbols(f'S_b_kVA_{name},U_b_{name}', real=True) # params



    ## per unit

    i_sd_pu = i_sd/I_bdq; # input in SI that is coverted to pu
    i_sq_pu = i_sq/I_bdq; # input in SI that is coverted to pu
    v_sd_pu = v_sd/V_bdq; # input in SI that is coverted to pu
    v_sq_pu = v_sq/V_bdq; # input in SI that is coverted to pu
    i_td_pu = i_td/I_bdq; # input in SI that is coverted to pu
    i_tq_pu = i_tq/I_bdq; # input in SI that is coverted to pu



    ## PLL #########################################################################


    # CTRL4 #########################################################################
    ## parameters:
    params_ctrl_4 = {}
    for item in ['T_vpoi','K_vpoi','T_f','K_f']:
        params_ctrl_4.update({f'{item}_{name}':vsg[item]})
        exec(f"{item} = sym.Symbol('{item}_{name}', real=True)")
    ## inputs:
    u_ctrl_4 = {}
    for item in ['v_s_ref','omega_ref','p_r','q_r']:
        u_ctrl_4.update({f'{item}_{name}':vsg[item]})
        exec(f"{item} = sym.Symbol('{item}_{name}', real=True)")
    ## dynamic states:
    x_list_ctrl_4 = ['omega_v_filt','v_s_filt']
    for item in x_list_ctrl_4:
        exec(f"{item} = sym.Symbol('{item}_{name}', real=True)")
    ## algebraic states
    y_list_ctrl_4 = ['p_m_ref','q_s_ref']
    for item in y_list_ctrl_4:
        exec(f"{item} = sym.Symbol('{item}_{name}', real=True)")    
    ## feedbacks:
    feedbacks = ['omega_v']
    for item in feedbacks:
        exec(f"{item} = sym.Symbol('{item}_{name}', real=True)")       

    domega_v_filt = 1/T_f*(omega_v - omega_v_filt)
    dv_s_filt = 1/T_vpoi*((v_sd_pu**2+v_sq_pu**2)**0.5 - v_s_filt)

    eq_p_m_ref = -p_m_ref + p_r + K_f*(omega_ref - omega_v_filt) + p_sec[name]   # PFR and secondary input 
    eq_q_s_ref = -q_s_ref + q_r + K_vpoi*(v_s_ref - v_s_filt)


    # from derivatives to the integrator
    f_ctrl_4 = [domega_v_filt,dv_s_filt];
    x_ctrl_4 = [ omega_v_filt, v_s_filt];
    g_ctrl_4 = [ eq_p_m_ref, eq_q_s_ref] #eq_i_sd_ref, eq_i_sq_ref, ]
    y_ctrl_4 = [    p_m_ref,    q_s_ref] #   i_sd_ref,    i_sq_ref,    omega_v,    DV_sat];
   
    

    # CTRL3 #########################################################################
    ## parameters:
    params_ctrl_3 = {}
    for item in ['K_p','T_p','K_q','T_q','R_v','X_v','S_b_kVA','U_b','K_phi','H','D']:
        params_ctrl_3.update({f'{item}_{name}':vsg[item]})
        exec(f"{item} = sym.Symbol('{item}_{name}', real=True)")
    ## inputs:
    u_ctrl_3 = {}
    for item in ['p_m_ref','q_s_ref']: # []
        u_ctrl_3.update({f'{item}_{name}':vsg[item]})
        exec(f"{item} = sym.Symbol('{item}_{name}', real=True)")
    ## dynamic states:
    x_list_ctrl_3 = ['phi','omega_v','xi_q','omega_rads']
    for item in x_list_ctrl_3:
        exec(f"{item} = sym.Symbol('{item}_{name}', real=True)")
    ## algebraic states
    y_list_ctrl_3 = ['DV_sat','p_s_pu','q_s_pu']
    for item in y_list_ctrl_3:
        exec(f"{item} = sym.Symbol('{item}_{name}', real=True)")    
    ## feedbacks:   
    v_dc,dum = sym.symbols(f'v_dc_{name},dum_{name}', real=True)



    fault_flag = 0;
    e_0 = 1.0;    
    eq_p_s_pu = -p_s_pu + i_sd_pu*v_sd_pu + i_sq_pu*v_sq_pu; # pu
    eq_q_s_pu = -q_s_pu + i_sd_pu*v_sq_pu - i_sq_pu*v_sd_pu; # pu 

    # from the integrator to the states
    epsilon_p = (p_m_ref - p_s_pu)*(1.0-fault_flag);
    epsilon_q =  q_s_ref - q_s_pu;

    if vsg['type'] == 'uvsg_i':
        eq_omega_v = -omega_v + K_p*(epsilon_p + xi_p/T_p) + 1.0; 

        dphi =  Omega_b*(omega_v-1.0) - K_phi*phi;
        dxi_p = epsilon_p;
        dxi_q = epsilon_q; 
        domega_rads = 1.0/1.0*(Omega_b*omega_v - omega_rads);

    if vsg['type'] == 'sm2_i':
        dphi =  Omega_b*(omega_v-omega_coi) - K_phi*phi;
        domega_v = 1/(2*H)*(p_m_ref - p_s_pu - D*(omega_v - 1.0))
        dxi_q = epsilon_q; 
        domega_rads = 1.0/1.0*(Omega_b*omega_v - omega_rads);

    DV =   K_q*(epsilon_q + xi_q/T_q); 

    eq_DV_sat = DV_sat - DV;
    #if DV_sat > 0.1 
    #    DV_sat = 0.1;
    #    dxi_q = 0.0;
    #end
    #if DV_sat < -0.1 
    #    DV_sat = -0.1;
    #    dxi_q = 0.0;
    #end   

    e = e_0 + DV_sat;   
    
    v_t_d_pu = 0.0
    v_t_q_pu = e

    v_t_d = (v_t_d_pu - R_v*i_sd_pu + X_v*i_sq_pu)*V_bdq
    v_t_q = (v_t_q_pu - R_v*i_sq_pu - X_v*i_sd_pu)*V_bdq
    
    #v_t_d = v_t_d_pu*V_bdq
    #v_t_q = v_t_q_pu*V_bdq
    
    eta_d_ref = v_t_d/v_dc*2
    eta_q_ref = v_t_q/v_dc*2

#    i_sd_pu = -(X_v*e + R_v*v_sd_pu + X_v*v_sq_pu)/(R_v**2 + X_v**2);
#    i_sq_pu = -(R_v*e + R_v*v_sq_pu - X_v*v_sd_pu)/(R_v**2 + X_v**2);
#    eq_i_sd_ref = -i_sd_ref + i_sd_pu*I_bdq
#    eq_i_sq_ref = -i_sq_ref + i_sq_pu*I_bdq

    #v_sd_ref_pu =    - R_v*i_sd_pu + X_v*i_sq_pu; 
    #v_sq_ref_pu = -e - R_v*i_sq_pu - X_v*i_sd_pu;
    #eq_v_sd_ref = -v_sd_ref + v_sd_ref_pu*V_bdq
    #eq_v_sq_ref = -v_sq_ref + v_sq_ref_pu*V_bdq

    # from derivatives to the integrator
    f_ctrl_3 = [dphi,domega_v,dxi_q,domega_rads];
    x_ctrl_3 = [ phi, omega_v, xi_q, omega_rads];
    g_ctrl_3 = [ eq_DV_sat, eq_p_s_pu, eq_q_s_pu] #eq_i_sd_ref, eq_i_sq_ref, ]
    y_ctrl_3 = [    DV_sat,    p_s_pu,    q_s_pu] #   i_sd_ref,    i_sq_ref,    omega_v,    DV_sat];
    x_0_ctrl_3 = [ 0.0, 0.0, 0.0, 2*np.pi*50]
    y_0_ctrl_3 = [ ] #     0.0,       V_bdq,          1]    



    # CTRL0 #########################################################################

    ## inputs:
    params_ctrl_0 = {}
    u_ctrl_0 = {}
    for item in []: # ['eta_d_ref','eta_q_ref','phi']: #+['eta_D','eta_Q']:
        u_ctrl_0.update({f'{item}_{name}':vsg[item]})
        exec(f"{item} = sym.Symbol('{item}_{name}', real=True)")
    ## algebraic states
    y_list_ctrl_0 = ['eta_d','eta_q'] + ['eta_D','eta_Q'] 
    for item in y_list_ctrl_0:
        exec(f"{item} = sym.Symbol('{item}_{name}', real=True)")        

    eq_eta_d = eta_d - eta_d_ref
    eq_eta_q = eta_q - eta_q_ref

    eq_eta_D = -eta_D + eta_d*cos(phi) - eta_q*sin(phi) # comment for test 1 
    eq_eta_Q = -eta_Q + eta_d*sin(phi) + eta_q*cos(phi) # comment for test 1

    # from derivatives to the integrator
    f_ctrl_0 = [];
    x_ctrl_0 = [];
    g_ctrl_0 = [eq_eta_d,eq_eta_q,eq_eta_D,eq_eta_Q]
    y_ctrl_0 = [   eta_d,   eta_q,   eta_D,   eta_Q];
    x_0_ctrl_0 = [ ]
    y_0_ctrl_0 = [0.0,  0.8, 0.0,  0.8]    



    ## VSC and Filters #########################################################################
    ## parameters:
    params_vsc_lcl = {}
    for item in ['L_t','R_t','C_m','L_s','R_s','omega','G_d']:
        params_vsc_lcl.update({f'{item}_{name}':vsg[item]})
        exec(f"{item} = sym.Symbol('{item}_{name}', real=True)")
    ## inputs:
    u_vsc_lcl = {}
    for item in ['v_dc']: #+['eta_D','eta_Q']:
        u_vsc_lcl.update({f'{item}_{name}':vsg[item]})
        exec(f"{item} = sym.Symbol('{item}_{name}', real=True)")
    ## dynamic states:
    x_list_vsc_lcl = []
    for item in x_list_vsc_lcl:
        exec(f"{item} = sym.Symbol('{item}_{name}', real=True)")
    ## algebraic states
    y_list_vsc_lcl = ['i_tD','i_tQ'] + ['v_mD','v_mQ'] + ['i_sD','i_sQ'] + ['v_sD','v_sQ']
    for item in y_list_vsc_lcl:
        exec(f"{item} = sym.Symbol('{item}_{name}', real=True)")    
    ## feedbacks:
    v_poiD,v_poiQ = sym.symbols(f'v_{bus}_D,v_{bus}_Q', real=True)
    i_poiD,i_poiQ = sym.symbols(f'i_{bus}_D,i_{bus}_Q', real=True)
    
    
    
    #eta_D = eta_D_ref # - Gv_in*(i_tD - i_sD)
    #eta_Q = eta_Q_ref #- Gv_in*(i_tQ - i_sQ)

    # LCL filter
    di_tD = 1/L_t*(eta_D/2*v_dc - R_t*i_tD + omega*L_t*i_tQ - v_mD)  
    di_tQ = 1/L_t*(eta_Q/2*v_dc - R_t*i_tQ - omega*L_t*i_tD - v_mQ) 
    dv_mD = 1/C_m*(i_tD + C_m*omega*v_mQ - G_d*v_mD - i_sD) 
    dv_mQ = 1/C_m*(i_tQ - C_m*omega*v_mD - G_d*v_mQ - i_sQ) 
    di_sD = 1/L_s*(v_mD - R_s*i_sD + omega*L_s*i_sQ - v_sD)  
    di_sQ = 1/L_s*(v_mQ - R_s*i_sQ - omega*L_s*i_sD - v_sQ) 
    
    # Grid interaction
    eq_i_poiD =  i_sD - i_poiD
    eq_i_poiQ =  i_sQ - i_poiQ   
    
    eq_v_sD =  v_sD - v_poiD
    eq_v_sQ =  v_sQ - v_poiQ    
    
    grid_dae['params'].pop(f'i_{bus}_D')
    grid_dae['params'].pop(f'i_{bus}_Q')

    # DAE
    f_vsc_lc = []
    x_vsc_lc = []
    g_vsc_lc = [di_tD, di_tQ, dv_mD, dv_mQ, di_sD, di_sQ, eq_i_poiD, eq_i_poiQ, eq_v_sD, eq_v_sQ]
    y_vsc_lc = [ i_tD,  i_tQ,  v_mD,  v_mQ,  i_sD,  i_sQ,    i_poiD,    i_poiQ,    v_sD,    v_sQ]    
    x_0_vsc_lc = [ ]
    y_0_vsc_lc = [ 0.0,  0.0,   0.0, V_bdq,     0,     0,         0,         0,       0,   V_bdq]
    
    
    ## Model integration
    f_vsg += f_vsc_lc + f_ctrl_0 + f_ctrl_3 + f_ctrl_4
    x_vsg += x_vsc_lc + x_ctrl_0 + x_ctrl_3 + x_ctrl_4
    g_vsg += g_vsc_lc + g_ctrl_0 + g_aux + g_ctrl_3 + g_ctrl_4  
    y_vsg += y_vsc_lc + y_ctrl_0 + y_aux + y_ctrl_3 + y_ctrl_4
    params_vsg.update(params_vsc_lcl)
    params_vsg.update(params_ctrl_0)
    params_vsg.update(params_ctrl_3)
    params_vsg.update(params_ctrl_4)
    u_vsg.update(u_vsc_lcl)
    u_vsg.update(u_ctrl_0)
    u_vsg.update(u_ctrl_3)
    u_vsg.update(u_ctrl_4)    
    
    h_vsg.update({f'i_sD_{name}':i_sD,f'i_sQ_{name}':i_sQ})

   # x_0_vsg += x_0_vsc_lc 
   # y_0_vsg += y_0_vsc_lc 
    
f_vsg += f_ctrl_5
x_vsg += x_ctrl_5
params_vsg.update(params_ctrl_5)
u_vsg.update(u_ctrl_5)